In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
import datetime as dt

# Limpieza

In [81]:
# cargamos los datos

with open('clima.json', 'r') as f:
    clima_json = json.load(f)


In [82]:
clima_json.keys()

dict_keys(['Elche', 'Ottawa', 'Adelaide', 'Helsinki'])

Los datos están en un diccionario donde los keys son las ciudades, y los values son listas de strings. Cada string contiene los datos de un mes.

In [83]:
def crear_dataframes(json):
    
    dicc_df = {}
    
    # iteramos por el diccionario contenido en el json
    for k, v in json.items():
        
        dicc_df[k] = []
        
        lista_df = []
        lista_faltas = []
        año = 2001
        
        # iteramos por cada string contenido en la lista de valores
        # cada elemento es un string de los datos de un mes
        for i in v:
            # dividimos los datos por los saltos de linea
            datos = i.split('\n')

            # definimos los nombres de las columnas (del dataframe que se va a crear) como la primera string de la nueva lista de datos
            columnas = datos.pop(0).split(')')
            for indice, columna in enumerate(columnas):
                columnas[indice] = columna.strip().replace(' ', '_') + ')'
            columnas.pop(-1)
            
            # creamos un diccionario donde los keys son los nombres de las columnas 
            dicc_datos = {}
            dicc_datos['dias'] = []
            for col in columnas:
                dicc_datos[col] = []
            
            mes = datos[0]
            if mes == "Jan":
                año += 1
                   
            if len(datos) < 203:
                lista_faltas.append((año, mes))

                pass
            else:
                # iteramos por una lista de posibles valores de la longitud de un mes
                # usamos este valor para sacar los datos segun los indices
                for n in range(27,34):
                    if datos[n] == 'Max Avg Min':
                        for x, key in enumerate(dicc_datos.keys()):
                            dicc_datos[key] = datos[x*n:x*n+n]
                # creamos un dataframe de los datos
                df = pd.DataFrame(dicc_datos)
                
                

                # creamos una copia del dataframe
                df2 = df.copy()
                for col in df2.columns[1:-1]:
                    try:
                        df2[col] = df2[col].str.split(' ')

                        # display(df2)

                    except:
                        print('error split:', k, mes, año)
                
                # dividimos las columnas que tienen datos combinados, usando la copia del dataframe
                # para sacar los nombres de las columnas 
                try:
                    for col in df.columns[1:-1]:
                        col1 = f'{col}_{df2.loc[0,col][0]}'
                        col2 = f'{col}_{df2.loc[0,col][1]}'
                        col3 = f'{col}_{df2.loc[0,col][2]}'
                        df[[col1, col2, col3]] = df[col].str.split(' ', expand=True)
                        df.drop(col, axis=1, inplace=True)
                except:
                    print('error columnas:', k, mes, año)
                    
                # eliminamos la primera fila que ahora es redundante y ponemos la columna de dias como índice    
                df.drop(0, axis=0, inplace=True)
                df['Month'] = mes
                df['Year'] = año
                df['Date'] =  df['dias'] + "-" + mes + "-" + str(año)
                # df.set_index('dias', inplace=True)
                
                lista_df.append(df)
                if mes == 'Dec':
                    df_año = pd.concat(lista_df, axis=0)
                    # año += 1
                    # display(df_año)
                    dicc_df[k].append(df_año)
        
    print(f"Meses que faltan: {lista_faltas}")            
    return dicc_df

In [84]:
dicc_datos = crear_dataframes(clima_json)

Meses que faltan: [(2004, 'Jan')]


In [85]:
dicc_datos.keys()

dict_keys(['Elche', 'Ottawa', 'Adelaide', 'Helsinki'])

In [86]:
dicc_datos['Elche'][0].head(2)

,dias,Precipitation_(in),Time_Temperature_(°F)_Max,Time_Temperature_(°F)_Avg,Time_Temperature_(°F)_Min,Dew_Point_(°F)_Max,Dew_Point_(°F)_Avg,Dew_Point_(°F)_Min,Humidity_(%)_Max,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(mph)_Max,Wind_Speed_(mph)_Avg,Wind_Speed_(mph)_Min,Pressure_(in)_Max,Pressure_(in)_Avg,Pressure_(in)_Min,Month,Year,Date
1,1,0.00,57,54.9,54,55,47.3,41,94,75.9,62,23,15.6,12,30.2,30.2,30.1,Jan,2002,1-Jan-2002
2,2,0.00,57,54.9,50,52,47.8,45,82,76.7,67,25,14.3,3,30.2,30.1,30.1,Jan,2002,2-Jan-2002


In [87]:
for k, v in dicc_datos.items():
    dicc_datos[k] = pd.concat(v, axis=0)

In [88]:
for k, v in dicc_datos.items():
    v['Ciudad'] = k

In [89]:
for k, v in dicc_datos.items():
    v.to_csv(f'{k}.csv')

In [90]:
df_final = pd.concat([dicc_datos['Elche'], dicc_datos['Adelaide'],
                     dicc_datos['Helsinki'], dicc_datos['Ottawa']], axis=0)

In [91]:
df_final['Date'] = df_final['Date'].apply(pd.to_datetime)

In [92]:
df_group = df_final.groupby(['Month', 'Year']).count()
df_group

dias  Precipitation_(in)  Time_Temperature_(°F)_Max  \
Month Year                                                        
Apr   2002  2520                2520                       2520   
      2003  2400                2400                       2400   
      2004  2280                2280                       2280   
      2005  2160                2160                       2160   
      2006  2040                2040                       2040   
...          ...                 ...                        ...   
Sep   2018   600                 600                        600   
      2019   480                 480                        480   
      2020   360                 360                        360   
      2021   240                 240                        240   
      2022   120                 120                        120   

            Time_Temperature_(°F)_Avg  Time_Temperature_(°F)_Min  \
Month Year                                                         
Apr   2002                       2520                       2520   
      2003                       2400                       2400   
      2004                       2280                       2280   
      2005                       2160                       2160   
      2006                       2040                       2040   
...                               ...                        ...   
Sep   2018                        600                        600   
      2019                        480                        480   
      2020                        360                        360   
      2021                        240                        240   
      2022                        120                        120   

            Dew_Point_(°F)_Max  Dew_Point_(°F)_Avg  Dew_Point_(°F)_Min  \
Month Year                                                               
Apr   2002                2520                2520                2520   
      2003                2400                2400                2400   
      2004                2280                2280                2280   
      2005                2160                2160                2160   
      2006                2040                2040                2040   
...                        ...                 ...                 ...   
Sep   2018                 600                 600                 600   
      2019                 480                 480                 480   
      2020                 360                 360                 360   
      2021                 240                 240                 240   
      2022                 120                 120                 120   

            Humidity_(%)_Max  Humidity_(%)_Avg  Humidity_(%)_Min  \
Month Year                                                         
Apr   2002              2520              2520              2520   
      2003              2400              2400              2400   
      2004              2280              2280              2280   
      2005              2160              2160              2160   
      2006              2040              2040              2040   
...                      ...               ...               ...   
Sep   2018               600               600               600   
      2019               480               480               480   
      2020               360               360               360   
      2021               240               240               240   
      2022               120               120               120   

            Wind_Speed_(mph)_Max  Wind_Speed_(mph)_Avg  Wind_Speed_(mph)_Min  \
Month Year                                                                     
Apr   2002                  2520                  2520                  2520   
      2003                  2400                  2400                  2400   
      2004                  2280                  2280                  2280   
      2

In [93]:
df_final.head(2)

,dias,Precipitation_(in),Time_Temperature_(°F)_Max,Time_Temperature_(°F)_Avg,Time_Temperature_(°F)_Min,Dew_Point_(°F)_Max,Dew_Point_(°F)_Avg,Dew_Point_(°F)_Min,Humidity_(%)_Max,Humidity_(%)_Avg,...,Wind_Speed_(mph)_Max,Wind_Speed_(mph)_Avg,Wind_Speed_(mph)_Min,Pressure_(in)_Max,Pressure_(in)_Avg,Pressure_(in)_Min,Month,Year,Date,Ciudad
1,1,0.00,57,54.9,54,55,47.3,41,94,75.9,...,23,15.6,12,30.2,30.2,30.1,Jan,2002,2002-01-01,Elche
2,2,0.00,57,54.9,50,52,47.8,45,82,76.7,...,25,14.3,3,30.2,30.1,30.1,Jan,2002,2002-01-02,Elche


In [94]:
df_final.shape

(336462, 21)

In [96]:
df_final.dtypes

dias                                 object
Precipitation_(in)                   object
Time_Temperature_(°F)_Max            object
Time_Temperature_(°F)_Avg            object
Time_Temperature_(°F)_Min            object
Dew_Point_(°F)_Max                   object
Dew_Point_(°F)_Avg                   object
Dew_Point_(°F)_Min                   object
Humidity_(%)_Max                     object
Humidity_(%)_Avg                     object
Humidity_(%)_Min                     object
Wind_Speed_(mph)_Max                 object
Wind_Speed_(mph)_Avg                 object
Wind_Speed_(mph)_Min                 object
Pressure_(in)_Max                    object
Pressure_(in)_Avg                    object
Pressure_(in)_Min                    object
Month                                object
Year                                  int64
Date                         datetime64[ns]
Ciudad                               object
dtype: object

In [97]:
columnas_num = ['Precipitation_(in)', 'Time_Temperature_(°F)_Max',
       'Time_Temperature_(°F)_Avg', 'Time_Temperature_(°F)_Min',
       'Dew_Point_(°F)_Max', 'Dew_Point_(°F)_Avg', 'Dew_Point_(°F)_Min',
       'Humidity_(%)_Max', 'Humidity_(%)_Avg', 'Humidity_(%)_Min',
       'Wind_Speed_(mph)_Max', 'Wind_Speed_(mph)_Avg', 'Wind_Speed_(mph)_Min',
       'Pressure_(in)_Max', 'Pressure_(in)_Avg', 'Pressure_(in)_Min']

In [98]:
for i in columnas_num:
    df_final[i] = df_final[i].astype('float64')

In [99]:
def far_to_cel(columna):
    try:
        return round(((columna - 32) * (5/9)), 1)
    except:
        print(columna)
        return columna

In [100]:
df_final.columns

Index(['dias', 'Precipitation_(in)', 'Time_Temperature_(°F)_Max',
       'Time_Temperature_(°F)_Avg', 'Time_Temperature_(°F)_Min',
       'Dew_Point_(°F)_Max', 'Dew_Point_(°F)_Avg', 'Dew_Point_(°F)_Min',
       'Humidity_(%)_Max', 'Humidity_(%)_Avg', 'Humidity_(%)_Min',
       'Wind_Speed_(mph)_Max', 'Wind_Speed_(mph)_Avg', 'Wind_Speed_(mph)_Min',
       'Pressure_(in)_Max', 'Pressure_(in)_Avg', 'Pressure_(in)_Min', 'Month',
       'Year', 'Date', 'Ciudad'],
      dtype='object')

In [101]:
for i in ['Time_Temperature_(°F)_Max',
       'Time_Temperature_(°F)_Avg', 'Time_Temperature_(°F)_Min',
       'Dew_Point_(°F)_Max', 'Dew_Point_(°F)_Avg', 'Dew_Point_(°F)_Min']:
    
    df_final[i] = df_final[i].apply(far_to_cel)

In [102]:
for col in df_final.columns:
    print(f"'{col}': '{col}', ")

'dias': 'dias', 
'Precipitation_(in)': 'Precipitation_(in)', 
'Time_Temperature_(°F)_Max': 'Time_Temperature_(°F)_Max', 
'Time_Temperature_(°F)_Avg': 'Time_Temperature_(°F)_Avg', 
'Time_Temperature_(°F)_Min': 'Time_Temperature_(°F)_Min', 
'Dew_Point_(°F)_Max': 'Dew_Point_(°F)_Max', 
'Dew_Point_(°F)_Avg': 'Dew_Point_(°F)_Avg', 
'Dew_Point_(°F)_Min': 'Dew_Point_(°F)_Min', 
'Humidity_(%)_Max': 'Humidity_(%)_Max', 
'Humidity_(%)_Avg': 'Humidity_(%)_Avg', 
'Humidity_(%)_Min': 'Humidity_(%)_Min', 
'Wind_Speed_(mph)_Max': 'Wind_Speed_(mph)_Max', 
'Wind_Speed_(mph)_Avg': 'Wind_Speed_(mph)_Avg', 
'Wind_Speed_(mph)_Min': 'Wind_Speed_(mph)_Min', 
'Pressure_(in)_Max': 'Pressure_(in)_Max', 
'Pressure_(in)_Avg': 'Pressure_(in)_Avg', 
'Pressure_(in)_Min': 'Pressure_(in)_Min', 
'Month': 'Month', 
'Year': 'Year', 
'Date': 'Date', 
'Ciudad': 'Ciudad', 


In [103]:
df_final['Precipitation_(in)'] = df_final['Precipitation_(in)'].apply(lambda x: x * 2.54)

In [104]:
for i in ['Pressure_(in)_Max', 'Pressure_(in)_Avg', 'Pressure_(in)_Min']:
    df_final[i] = df_final[i].apply(lambda x: round(x * 33.86389))

In [105]:
for i in ['Wind_Speed_(mph)_Max', 'Wind_Speed_(mph)_Avg', 'Wind_Speed_(mph)_Min']:
    df_final[i] = df_final[i].apply(lambda y: round(y * 1.609344, 1))

In [106]:
df_final.rename(columns = {'dias': 'Day', 
                            'Precipitation_(in)': 'Precipitation_(mm)', 
                            'Time_Temperature_(°F)_Max': 'Temperature_(°C)_Max', 
                            'Time_Temperature_(°F)_Avg': 'Temperature_(°C)_Avg', 
                            'Time_Temperature_(°F)_Min': 'Temperature_(°C)_Min', 
                            'Dew_Point_(°F)_Max': 'Dew_Point_(°C)_Max', 
                            'Dew_Point_(°F)_Avg': 'Dew_Point_(°C)_Avg', 
                            'Dew_Point_(°F)_Min': 'Dew_Point_(°C)_Min', 
                            'Humidity_(%)_Max': 'Humidity_(%)_Max', 
                            'Humidity_(%)_Avg': 'Humidity_(%)_Avg', 
                            'Humidity_(%)_Min': 'Humidity_(%)_Min', 
                            'Wind_Speed_(mph)_Max': 'Wind_Speed_(kph)_Max', 
                            'Wind_Speed_(mph)_Avg': 'Wind_Speed_(kph)_Avg', 
                            'Wind_Speed_(mph)_Min': 'Wind_Speed_(kph)_Min', 
                            'Pressure_(in)_Max': 'Pressure_(hPa)_Max', 
                            'Pressure_(in)_Avg': 'Pressure_(hPa)_Avg', 
                            'Pressure_(in)_Min': 'Pressure_(hPa)_Min', 
                            'Month': 'Month', 
                            'Year': 'Year', 
                            'Date': 'Date', 
                            'Ciudad': 'City'}, inplace = True)

In [107]:
df_final['Year'] = df_final['Date'].apply(lambda x: x.strftime('%Y'))

In [108]:
df_final['Month'] = df_final['Date'].apply(lambda x: x.strftime('%B'))

In [109]:
df_final['Day'] = df_final['Date'].apply(lambda x: x.strftime('%-d'))

In [110]:
def estacion(x, ciudad):
    if ciudad == 'Adelaide':
        if x.strftime('%m-%d') <= '03-20':
            return 'summer'
        elif x.strftime('%m-%d') <= '06-20':
            return 'autumn'
        elif x.strftime('%m-%d') <= '09-20' :
            return 'winter'
        elif x.strftime('%m-%d') <= '12-20':
            return 'spring'
        elif x.strftime('%m-%d') <= '12-31':
            return 'summer'
    else:
        if x.strftime('%m-%d') <= '03-20':
            return 'winter'
        elif x.strftime('%m-%d') <= '06-20':
            return 'spring'
        elif x.strftime('%m-%d') <= '09-20' :
            return 'summer'
        elif x.strftime('%m-%d') <= '12-20':
            return 'autumn'
        elif x.strftime('%m-%d') <= '12-31':
            return 'winter'

In [111]:
df_final['Season'] = df_final.apply(lambda x: estacion(x['Date'], x['City']), axis=1)

In [112]:
df_final.isnull().sum()

Day                     0
Precipitation_(mm)      0
Temperature_(°C)_Max    0
Temperature_(°C)_Avg    0
Temperature_(°C)_Min    0
Dew_Point_(°C)_Max      0
Dew_Point_(°C)_Avg      0
Dew_Point_(°C)_Min      0
Humidity_(%)_Max        0
Humidity_(%)_Avg        0
Humidity_(%)_Min        0
Wind_Speed_(kph)_Max    0
Wind_Speed_(kph)_Avg    0
Wind_Speed_(kph)_Min    0
Pressure_(hPa)_Max      0
Pressure_(hPa)_Avg      0
Pressure_(hPa)_Min      0
Month                   0
Year                    0
Date                    0
City                    0
Season                  0
dtype: int64

In [113]:
df_final.columns

Index(['Day', 'Precipitation_(mm)', 'Temperature_(°C)_Max',
       'Temperature_(°C)_Avg', 'Temperature_(°C)_Min', 'Dew_Point_(°C)_Max',
       'Dew_Point_(°C)_Avg', 'Dew_Point_(°C)_Min', 'Humidity_(%)_Max',
       'Humidity_(%)_Avg', 'Humidity_(%)_Min', 'Wind_Speed_(kph)_Max',
       'Wind_Speed_(kph)_Avg', 'Wind_Speed_(kph)_Min', 'Pressure_(hPa)_Max',
       'Pressure_(hPa)_Avg', 'Pressure_(hPa)_Min', 'Month', 'Year', 'Date',
       'City', 'Season'],
      dtype='object')

In [114]:
reorder = ['City', 'Date', 'Day', 'Month', 'Year', 'Season', 'Precipitation_(mm)', 'Temperature_(°C)_Max',
       'Temperature_(°C)_Avg', 'Temperature_(°C)_Min', 'Dew_Point_(°C)_Max',
       'Dew_Point_(°C)_Avg', 'Dew_Point_(°C)_Min', 'Humidity_(%)_Max',
       'Humidity_(%)_Avg', 'Humidity_(%)_Min', 'Wind_Speed_(kph)_Max',
       'Wind_Speed_(kph)_Avg', 'Wind_Speed_(kph)_Min', 'Pressure_(hPa)_Max',
       'Pressure_(hPa)_Avg', 'Pressure_(hPa)_Min']

In [115]:
df_final = df_final.reindex(columns = reorder)

In [116]:
df_final.head()

,City,Date,Day,Month,Year,Season,Precipitation_(mm),Temperature_(°C)_Max,Temperature_(°C)_Avg,Temperature_(°C)_Min,...,Dew_Point_(°C)_Min,Humidity_(%)_Max,Humidity_(%)_Avg,Humidity_(%)_Min,Wind_Speed_(kph)_Max,Wind_Speed_(kph)_Avg,Wind_Speed_(kph)_Min,Pressure_(hPa)_Max,Pressure_(hPa)_Avg,Pressure_(hPa)_Min
1,Elche,2002-01-01,1,January,2002,winter,0.0,13.9,12.7,12.2,...,5.0,94.0,75.9,62.0,37.0,25.1,19.3,1023,1023,1019
2,Elche,2002-01-02,2,January,2002,winter,0.0,13.9,12.7,10.0,...,7.2,82.0,76.7,67.0,40.2,23.0,4.8,1023,1019,1019
3,Elche,2002-01-03,3,January,2002,winter,0.0,13.9,11.6,8.9,...,7.2,94.0,83.8,72.0,14.5,5.6,0.0,1019,1016,1013
4,Elche,2002-01-04,4,January,2002,winter,0.0,15.0,12.5,8.9,...,6.1,94.0,77.6,63.0,29.0,12.9,3.2,1023,1016,1013
5,Elche,2002-01-05,5,January,2002,winter,0.0,13.9,13.0,12.2,...,5.0,88.0,68.2,58.0,27.4,23.3,9.7,1029,1026,1023


In [118]:
df_final.to_csv('clima_2002-2022.csv')